In [87]:
# Import the necessary libraries 
# Recall highly correlated column (base and exercised options value) is our highly corr. column
# used for manipulating directory paths
import os

# Scientific and vector computation for python
import numpy as np
import pandas as pd

# Plotting library
from matplotlib import pyplot

# import function to split data into train and test dfs
from sklearn.model_selection import train_test_split

# Optimization module in scipy
from scipy import optimize

# Import logistic regression model from SK Learn
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

# Import glob as glob, used to consolidate data files
import glob

In [88]:
# read all csv files within the test_data folder. This should be 2008-2018 (11 years), of DHS contract award data
all_files = glob.glob("../data/train_data/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, header = 0, usecols = ['contract_transaction_unique_key',
                        'contract_award_unique_key',
                        'federal_action_obligation','total_dollars_obligated', 'base_and_exercised_options_value', 
                        'current_total_value_of_award', 'base_and_all_options_value','potential_total_value_of_award',
                        'action_date'],
                 dtype = {'contract_transaction_unique_key':'str','contract_transaction_unique_key':'str',
                          'federal_action_obligation': 'float','total_dollars_obligated': 'float', 
                          'base_and_exercised_options_value': 'float', 'current_total_value_of_award': 'float', 
                          'base_and_all_options_value': 'float','potential_total_value_of_award': 'float',
                          'action_date':'str'})
    li.append(df)

df = pd.concat(li, axis=0, ignore_index=True)


In [89]:
# This is what we are using for our rate of Execution
df['Percent Cumulatively Obligated over potential total value of award'] = df['total_dollars_obligated'] / df['potential_total_value_of_award']

# Create Indicator Variable
df['Indicator'] = df['federal_action_obligation']<-1000

In [90]:
df = df.fillna(0)
# Convert action date to pd.datetime
df['action_date'] = pd.to_datetime(df['action_date'])
# set index to each contract and date
df.set_index(['contract_award_unique_key','action_date'], inplace = True)
df.sort_index()

contract_transaction_unique_key  \
contract_award_unique_key              action_date                                                   
CONT_AWD_00001_7022_HSFE0408A0017_7022 2008-02-15                7022_7022_00001_0_HSFE0408A0017_0   
CONT_AWD_00001_7022_HSFE0408A0035_7022 2008-02-20                7022_7022_00001_0_HSFE0408A0035_0   
CONT_AWD_00001_7022_HSFE0408A0066_7022 2008-05-15                7022_7022_00001_0_HSFE0408A0066_0   
CONT_AWD_00001_7022_HSFE0408A0067_7022 2008-03-31                7022_7022_00001_0_HSFE0408A0067_0   
                                       2012-07-27           7022_7022_00001_P00001_HSFE0408A0067_0   
...                                                                                            ...   
CONT_IDV_TPDFIGBPA100001_2036          2010-10-26   2036_-NONE-_TPDFIGBPA100001_T501_-NONE-_-NONE-   
                                       2010-10-26    2036_-NONE-_TPDFIGBPA100001_500_-NONE-_-NONE-   
CONT_IDV_WRO02005_7012                 2017-05-31          7012_-NONE-_WRO02005_XXX9_-NONE-_-NONE-   
CONT_IDV_WRO9812_7012                  2017-05-31           7012_-NONE-_WRO9812_XXX9_-NONE-_-NONE-   
CONT_IDV_WRO9816_7012                  2017-05-31           7012_-NONE-_WRO9816_XXX9_-NONE-_-NONE-   

                                                    federal_action_obligation  \
contract_award_unique_key              action_date                              
CONT_AWD_00001_7022_HSFE0408A0017_7022 2008-02-15                     15000.0   
CONT_AWD_00001_7022_HSFE0408A0035_7022 2008-02-20                     10000.0   
CONT_AWD_00001_7022_HSFE0408A0066_7022 2008-05-15                      5000.0   
CONT_AWD_00001_7022_HSFE0408A0067_7022 2008-03-31                      1500.0   
                                       2012-07-27                       -30.0   
...                                                                       ...   
CONT_IDV_TPDFIGBPA100001_2036          2010-10-26                         0.0   
                                       2010-10-26                         0.0   
CONT_IDV_WRO02005_7012                 2017-05-31                         0.0   
CONT_IDV_WRO9812_7012                  2017-05-31                         0.0   
CONT_IDV_WRO9816_7012                  2017-05-31                         0.0   

                                                    total_dollars_obligated  \
contract_award_unique_key              action_date                            
CONT_AWD_00001_7022_HSFE0408A0017_7022 2008-02-15                       0.0   
CONT_AWD_00001_7022_HSFE0408A0035_7022 2008-02-20                       0.0   
CONT_AWD_00001_7022_HSFE0408A0066_7022 2008-05-15                       0.0   
CONT_AWD_00001_7022_HSFE0408A0067_7022 2008-03-31                       0.0   
                                       2012-07-27                       0.0   
...                                                                     ...   
CONT_IDV_TPDFIGBPA100001_2036          2010-10-26                       0.0   
                                       2010-10-26                       0.0   
CONT_IDV_WRO02005_7012                 2017-05-31                       0.0   
CONT_IDV_WRO9812_7012                  2017-05-31                       0.0   
CONT_IDV_WRO9816_7012                  2017-05-31                       0.0   

                                                    base_and_exercised_options_value  \
contract_award_unique_key              action_date                                     
CONT_AWD_00001_7022_HSFE0408A0017_7022 2008-02-15                            15000.0   
CONT_AWD_00001_7022_HSFE0408A0035_7022 2008-02-20                            10000.0   
CONT_AWD_00001_7022_HSFE0408A0066_7022 2008-05-15                             5000.0   
CONT_AWD_00001_7022_HSFE0408A0067_7022 2008-03-31                             1500.0   
                                       2012-07-27                              -30.0   
...                                       

In [91]:
# We want to only focus on execution rate (for now). I'm not sure which column is the best indicator of execution rate
# we can follow up with Eric. I'm going to stick with Percent Cumulatively Obligated over potential total value of award
# as wequal to the execution rate.
drop_columns = [column for column in df.columns if column != 'Percent Cumulatively Obligated over potential total value of award']
df.drop(drop_columns,axis = 1, inplace=True)

df.rename(columns = {'Percent Cumulatively Obligated over potential total value of award':'execution_rate'},inplace = True)

In [92]:
df.head(4)

,,execution_rate
contract_award_unique_key,action_date,
CONT_AWD_HSBP20080015201839_7014_GS21F0031U_4730,2008-09-30,0.0
CONT_IDV_HSHQDC06C00065_7001,2008-09-30,0.0
CONT_AWD_HSCG2808P7ABEC9_7008_-NONE-_-NONE-,2008-09-30,0.0
CONT_IDV_HSHQDC08C00190_7001,2008-09-30,0.0


In [93]:
contracts = [contract for contract, date in list(df.index)]
unique_k = set(contracts)
print('This is the number of unique Contracts: ' + str(len(unique_k)))
print('This is the number of transactions: ' + str(len(contracts)))

This is the number of unique Contracts: 485094
This is the number of transactions: 1020089


# Selecting contracts w/ 5 or more transactions from 2008-2018

In [94]:
# I grouped the dataframe by contract and used the .count(method)
# to determine the number of entries. I then used an inequality to get a boolean series of the number of contracts 
# that satisfied the condition of at least 5 transactions.
test = df.groupby(level='contract_award_unique_key')['execution_rate'].count() >= 5
test = test[test==True]
# This is the list of contracts that have more than 5 transactions. 
highlighted_contracts = test.index



### Utilize FY19 Data as Test Data

In [95]:
filename = '../data/FY2019_070_Contracts_Full_20200110_1.csv'

In [96]:
test_df = pd.read_csv(filename, header = 0, usecols = ['contract_transaction_unique_key',
                        'contract_award_unique_key',
                        'federal_action_obligation','total_dollars_obligated', 'base_and_exercised_options_value', 
                        'current_total_value_of_award', 'base_and_all_options_value','potential_total_value_of_award',
                        'action_date'],
                 dtype = {'contract_transaction_unique_key':'str','contract_transaction_unique_key':'str',
                          'federal_action_obligation': 'float','total_dollars_obligated': 'float', 
                          'base_and_exercised_options_value': 'float', 'current_total_value_of_award': 'float', 
                          'base_and_all_options_value': 'float','potential_total_value_of_award': 'float',
                          'action_date':'str'})

In [97]:
print('This is the # of transactions is FY19 for DHS: ' + str(len(test_df)))

This is the # of transactions is FY19 for DHS: 66533


In [98]:
test_df = test_df.fillna(0)
# Convert action date to pd.datetime
test_df['action_date'] = pd.to_datetime(test_df['action_date'])
# set index to each contract and date
test_df.set_index(['contract_award_unique_key','action_date'], inplace = True)
test_df.sort_index()
fy19_df = test_df.copy()

In [99]:
# We want to only focus on execution rate (for now). I'm not sure which column is the best indicator of execution rate
# we can follow up with Eric. I'm going to stick with Percent Cumulatively Obligated over potential total value of award
# as wequal to the execution rate.
drop_columns = [column for column in test_df.columns if column != 'Percent Cumulatively Obligated over potential total value of award']
test_df.drop(drop_columns,axis = 1, inplace=True)

test_df.rename(columns = {'Percent Cumulatively Obligated over potential total value of award':'execution_rate'},inplace = True)

In [100]:
contracts = [contract for contract, date in list(test_df.index)]
unique_k_fy19 = set(contracts)
print('This is the number of unique Contracts: ' + str(len(unique_k_fy19)))
print('This is the number of transactions: ' + str(len(contracts)))

This is the number of unique Contracts: 44314
This is the number of transactions: 66533


In [101]:
type(unique_k_fy19)

set

In [102]:
highlighted_contracts = set(highlighted_contracts)

In [103]:
len(unique_k_fy19.intersection(highlighted_contracts))

4022

In [104]:
mask = list(unique_k_fy19.intersection(highlighted_contracts))

In [105]:
fy19_df.index.get_level_values('contract_award_unique_key').unique()

Index(['CONT_AWD_HSCEMS16F00057_7012_HSCEMS12A00018_7012',
       'CONT_AWD_HSFE2015J0271_7022_HSFE2013D0161_7022',
       'CONT_AWD_70Z02918PX0808600_7008_-NONE-_-NONE-',
       'CONT_AWD_70Z03818PW0000011_7008_-NONE-_-NONE-',
       'CONT_AWD_70Z08018PP45K8500_7008_-NONE-_-NONE-',
       'CONT_IDV_HSCG3816DL00004_7008',
       'CONT_AWD_70Z08418F4YF02400_7008_GS33F0009P_4730',
       'CONT_AWD_HSCG3211J300016_7008_HSCG3210DR00021_7008',
       'CONT_AWD_HSCG3212J300032_7008_HSCG3210DR00021_7008',
       'CONT_IDV_HSTS0514AMED077_7013',
       ...
       'CONT_AWD_70RSAT18C00000012_7001_-NONE-_-NONE-',
       'CONT_AWD_70US0919C70090057_7009_-NONE-_-NONE-',
       'CONT_AWD_70FA4019P00000011_7022_-NONE-_-NONE-',
       'CONT_AWD_HSBP1015J00091_7014_HSBP1015A00018_7014',
       'CONT_AWD_HSFE2013MP6330565_7022_-NONE-_-NONE-',
       'CONT_AWD_70Z04019F61427Y00_7008_HSCG4015D60744_7008',
       'CONT_AWD_70Z03819PD0000080_7008_-NONE-_-NONE-',
       'CONT_AWD_HSBP1015F00038_7014_HSBP101

In [106]:
highlighted_fy19 = fy19_df.loc[(list(highlighted_contracts), slice(None)),:]

In [107]:
highlighted_fy19.head()

,,contract_transaction_unique_key,federal_action_obligation,total_dollars_obligated,base_and_exercised_options_value,current_total_value_of_award,base_and_all_options_value,potential_total_value_of_award
contract_award_unique_key,action_date,,,,,,,
CONT_AWD_HSCEMS16F00057_7012_HSCEMS12A00018_7012,2018-10-19,7012_7012_HSCEMS16F00057_P00005_HSCEMS12A00018_0,-7702.86,6003953.42,-7702.86,6059478.42,-148339.0,6.067181e+06
CONT_IDV_HSCG3816DL00004_7008,2018-10-20,7008_-NONE-_HSCG3816DL00004_P00009_-NONE-_-NONE-,0.00,0.00,0.00,0.00,0.0,3.625838e+06
CONT_IDV_HSCEDM17D00009_7012,2018-10-02,7012_-NONE-_HSCEDM17D00009_P00008_-NONE-_-NONE-,0.00,0.00,0.00,0.00,0.0,4.573615e+08
CONT_IDV_HSCG3815D202051_7008,2018-10-02,7008_-NONE-_HSCG3815D202051_P00005_-NONE-_-NONE-,0.00,0.00,0.00,0.00,0.0,4.074217e+05
CONT_IDV_HSCG4016D60308_7008,2018-10-02,7008_-NONE-_HSCG4016D60308_P00007_-NONE-_-NONE-,0.00,0.00,0.00,0.00,0.0,1.818706e+07


In [108]:
highlighted_fy19.index.get_level_values(0).unique()

Index(['CONT_AWD_HSCEMS16F00057_7012_HSCEMS12A00018_7012',
       'CONT_IDV_HSCG3816DL00004_7008', 'CONT_IDV_HSCEDM17D00009_7012',
       'CONT_IDV_HSCG3815D202051_7008', 'CONT_IDV_HSCG4016D60308_7008',
       'CONT_IDV_HSFLAR16D00001_7015', 'CONT_IDV_HSTS0311DCIO391_7013',
       'CONT_AWD_HSSCCG14F00396_7003_GS03FPM011_4732',
       'CONT_AWD_HSSCCG14F00399_7003_GS03FPM006_4732',
       'CONT_AWD_HSSCCG14Z00017_7003_GS03FPM011_4732',
       ...
       'CONT_AWD_HSBP1012P00838_7014_-NONE-_-NONE-',
       'CONT_AWD_HSFE5015J0268_7022_HSFE5012D0245_7022',
       'CONT_AWD_HSBP1012P00975_7014_-NONE-_-NONE-',
       'CONT_AWD_HSFETX15F0020_7022_GS02F0048M_4730',
       'CONT_AWD_HSCG8513FP45M65_7008_HSCG2313DATB041_7008',
       'CONT_AWD_HSCEDM15P00082_7012_-NONE-_-NONE-',
       'CONT_AWD_70RFP218FREC00009_7001_HSHQEC16D00008_7001',
       'CONT_AWD_HSHQPD06P00002_7001_-NONE-_-NONE-',
       'CONT_AWD_HSHQDC10J00304_7001_HSHQDC06D00024_7001',
       'CONT_AWD_HSSCCG14F00581_7003_HSHQDC1

In [110]:
highlighted_fy19.drop('base_and_exercised_options_value',axis = 1, inplace = True)

In [111]:
highlighted_df = df.loc[(list(highlighted_contracts), slice(None)),:]

In [112]:
highlighted_fy19

,,contract_transaction_unique_key,federal_action_obligation,total_dollars_obligated,current_total_value_of_award,base_and_all_options_value,potential_total_value_of_award
contract_award_unique_key,action_date,,,,,,
CONT_AWD_HSCEMS16F00057_7012_HSCEMS12A00018_7012,2018-10-19,7012_7012_HSCEMS16F00057_P00005_HSCEMS12A00018_0,-7702.86,6003953.42,6059478.42,-148339.00,6.067181e+06
CONT_IDV_HSCG3816DL00004_7008,2018-10-20,7008_-NONE-_HSCG3816DL00004_P00009_-NONE-_-NONE-,0.00,0.00,0.00,0.00,3.625838e+06
CONT_IDV_HSCEDM17D00009_7012,2018-10-02,7012_-NONE-_HSCEDM17D00009_P00008_-NONE-_-NONE-,0.00,0.00,0.00,0.00,4.573615e+08
CONT_IDV_HSCG3815D202051_7008,2018-10-02,7008_-NONE-_HSCG3815D202051_P00005_-NONE-_-NONE-,0.00,0.00,0.00,0.00,4.074217e+05
CONT_IDV_HSCG4016D60308_7008,2018-10-02,7008_-NONE-_HSCG4016D60308_P00007_-NONE-_-NONE-,0.00,0.00,0.00,0.00,1.818706e+07
...,...,...,...,...,...,...,...
CONT_AWD_HSHQPD06P00002_7001_-NONE-_-NONE-,2018-10-01,7001_-NONE-_HSHQPD06P00002_P00009_-NONE-_0,0.00,147859.99,147859.99,0.00,1.478600e+05
CONT_AWD_HSHQDC10J00304_7001_HSHQDC06D00024_7001,2019-09-17,7001_7001_HSHQDC10J00304_P00008_HSHQDC06D00024_0,-448013.23,4977501.44,4977501.44,-448013.23,7.005469e+06
CONT_AWD_HSHQDC15F00108_7001_GS00Q14OADS124_4732,2019-07-09,7001_4732_HSHQDC15F00108_P00025_GS00Q14OADS124_0,505000.00,46688938.83,48905927.58,505000.00,4.890593e+07


In [117]:
# This is what we are using for our rate of Execution
highlighted_fy19['Percent Cumulatively Obligated over potential total value of award'] = highlighted_fy19['total_dollars_obligated'] / highlighted_fy19['potential_total_value_of_award']

# Create Indicator Variable
highlighted_fy19['Indicator'] = highlighted_fy19['federal_action_obligation']<-1000

C:\Users\belincoln\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\belincoln\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [119]:
highlighted_fy19.rename(columns = {'Percent Cumulatively Obligated over potential total value of award':'execution_rate'},inplace = True)

In [124]:
highlighted_fy19.drop(columns = [column for column in highlighted_fy19.columns if column != 'execution_rate'],inplace = True)

In [113]:
highlighted_fy19.index.get_level_values(0).unique()

Index(['CONT_AWD_HSCEMS16F00057_7012_HSCEMS12A00018_7012',
       'CONT_IDV_HSCG3816DL00004_7008', 'CONT_IDV_HSCEDM17D00009_7012',
       'CONT_IDV_HSCG3815D202051_7008', 'CONT_IDV_HSCG4016D60308_7008',
       'CONT_IDV_HSFLAR16D00001_7015', 'CONT_IDV_HSTS0311DCIO391_7013',
       'CONT_AWD_HSSCCG14F00396_7003_GS03FPM011_4732',
       'CONT_AWD_HSSCCG14F00399_7003_GS03FPM006_4732',
       'CONT_AWD_HSSCCG14Z00017_7003_GS03FPM011_4732',
       ...
       'CONT_AWD_HSBP1012P00838_7014_-NONE-_-NONE-',
       'CONT_AWD_HSFE5015J0268_7022_HSFE5012D0245_7022',
       'CONT_AWD_HSBP1012P00975_7014_-NONE-_-NONE-',
       'CONT_AWD_HSFETX15F0020_7022_GS02F0048M_4730',
       'CONT_AWD_HSCG8513FP45M65_7008_HSCG2313DATB041_7008',
       'CONT_AWD_HSCEDM15P00082_7012_-NONE-_-NONE-',
       'CONT_AWD_70RFP218FREC00009_7001_HSHQEC16D00008_7001',
       'CONT_AWD_HSHQPD06P00002_7001_-NONE-_-NONE-',
       'CONT_AWD_HSHQDC10J00304_7001_HSHQDC06D00024_7001',
       'CONT_AWD_HSSCCG14F00581_7003_HSHQDC1

In [126]:
highlighted_fy19

,,execution_rate
contract_award_unique_key,action_date,
CONT_AWD_HSCEMS16F00057_7012_HSCEMS12A00018_7012,2018-10-19,0.989579
CONT_IDV_HSCG3816DL00004_7008,2018-10-20,0.000000
CONT_IDV_HSCEDM17D00009_7012,2018-10-02,0.000000
CONT_IDV_HSCG3815D202051_7008,2018-10-02,0.000000
CONT_IDV_HSCG4016D60308_7008,2018-10-02,0.000000
...,...,...
CONT_AWD_HSHQPD06P00002_7001_-NONE-_-NONE-,2018-10-01,1.000000
CONT_AWD_HSHQDC10J00304_7001_HSHQDC06D00024_7001,2019-09-17,0.710516
CONT_AWD_HSHQDC15F00108_7001_GS00Q14OADS124_4732,2019-07-09,0.954668


In [114]:
highlighted_df

,,execution_rate
contract_award_unique_key,action_date,
CONT_IDV_HSHQDC06C00065_7001,2008-09-30,0.0
CONT_AWD_HSSS0108F0077_7009_GS35F0503M_4730,2008-09-30,0.0
CONT_AWD_HSFEEM05F0317_7022_GS35F4274D_4730,2008-09-30,0.0
CONT_AWD_HSCEC708F00050_7012_GS07F0306L_4730,2008-09-30,0.0
CONT_AWD_HSHQDC08J00375_7001_HSHQDC06D00048_7001,2008-09-30,0.0
...,...,...
CONT_AWD_70RFP118FRE200001_7001_HSHQE213D00001_7001,2017-10-01,1.0
CONT_AWD_70RFP418FRE700032_7001_HSHQC715D00001_7001,2017-10-01,1.0
CONT_AWD_70RFP118FRE200002_7001_HSHQE213D00001_7001,2017-10-01,1.0
